Alright, let's start fresh by defining a comprehensive schema for your Weaviate instance that includes `Article`, `Author`, `Notion`, and `Tag` classes. Then, we'll outline how to access your data from MinIO and ingest it into Weaviate accordingly.

### Step 1: Define the Schema

Given your requirements, here's a proposed schema:

1. **Article**: Holds the content of articles.
2. **Author**: Information about the authors.
3. **Notion**: Represents documents or notes from Notion.
4. **Tag**: Keywords or topics associated with articles.



In [5]:
import weaviate

client = weaviate.Client("http://cda-DESKTOP:8080")

# Delete existing classes to start fresh (handle with caution in production)
#client.schema.delete_class('Article')
#client.schema.delete_class('Author')

C:\Users\david\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\weaviate\warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


In [6]:
# This schema ingests exisiting data 
schema = {
    "classes": [
        {
            "class": "NotionArticle",
            "description": "Stores individual articles",
            "properties": [
                {"name": "title", "dataType": ["string"], "description": "Title of the article"},
                {"name": "content", "dataType": ["text"], "description": "Content of the article"},
                {"name": "publishedAt", "dataType": ["dateTime"], "description": "Publish date of the article"},
                {"name": "url", "dataType": ["string"], "description": "URL of the article"},
                {"name": "tags", "dataType": ["Tag[]"], "description": "Tags associated with the article"},
                {"name": "notionDocument", "dataType": ["Notion"], "description": "Reference to the Notion document this article is based on"},
                {"name": "author", "dataType": ["Author"], "description": "Author of the article"}
            ]
        },
        {
            "class": "NotionAuthor",
            "description": "Information about authors",
            "properties": [
                {"name": "name", "dataType": ["string"], "description": "Name of the author"},
                {"name": "bio", "dataType": ["text"], "description": "Short biography of the author"},
                {"name": "articles", "dataType": ["Article[]"], "description": "Articles written by the author"}
            ]
        },
        {
            "class": "Notion",
            "description": "Represents a Notion document",
            "properties": [
                {"name": "objectName", "dataType": ["string"], "description": "Name of the Notion document"},
                {"name": "lastModified", "dataType": ["dateTime"], "description": "Last modified date of the Notion document"},
                {"name": "content", "dataType": ["text"], "description": "Content of the Notion document"}
            ]
        },
        {
            "class": "Tag",
            "description": "Keywords or topics associated with an article",
            "properties": [
                {"name": "name", "dataType": ["string"], "description": "Name of the tag"},
                {"name": "articles", "dataType": ["Article[]"], "description": "Articles associated with this tag"}
            ]
        }
    ]
}


### Step 2: Create the Schema in Weaviate

Use the Weaviate Python SDK to create this schema:



In [7]:
client.schema.create(schema)

UnexpectedStatusCodeError: Create class! Unexpected status code: 422, with response body: {'error': [{'message': 'class name "NotionArticle" already exists'}]}.

### Step 3: Access Data from MinIO

Assuming you have the MinIO Python SDK set up, let's connect to your MinIO instance and download the JSON file containing your data.



In [ ]:
from minio import Minio
from minio.error import S3Error

minioClient = Minio(
    "cda-DESKTOP:9000",
    access_key="cda_cdaprod",
    secret_key="cda_cdaprod",
    secure=False
)

bucket_name = "cda-datasets"
object_name = "obsidian-notion-data-metadata.json"
local_file_path = "obsidian-notion-data-metadata.json"

try:
    minioClient.fget_object(bucket_name, object_name, local_file_path)
    print("File downloaded successfully")
except S3Error as e:
    print("Error:", e)


### Step 4: Ingest Data into Weaviate

This is a starting point. You'll need to adapt the data transformation and ingestion logic based on the actual structure of your JSON file and how you want to map it to the `Article`, `Author`, `Notion`, and `Tag` classes in Weaviate. Each object ingested into Weaviate needs to adhere to the schema you've defined, including linking articles to their respective authors, tags, and associated Notion documents.

After downloading the file, load the JSON data, and ingest it into Weaviate. Given the new schema, you'll likely need to parse and transform the data from your JSON to fit the schema properly. Here's a simplified example of how you might start this process for `Article` objects:



In [9]:
import json

# Assuming 'local_file_path' is your downloaded JSON file
with open("obsidian-notion-data-metadata.json") as f:
    data = json.load(f)

# Simplified example to ingest an article
# for item in data:
# # Transform and ingest your data here
# # This is a placeholder; you'll need to adapt it based on your actual JSON structure
# article = {
#     "title": item.get("objectName"),
#     "content": item.get("content"),
#     # Add other fields as per your JSON structure
# }
# client.data_object.create(data_object=article, class_name="Article")
# print("Ingested article:", article["title"])
    
# Loop through your data and create Article objects in Weaviate
for item in data:
    article_data = {
        "objectName": item.get("object_name"),
        "lastModified": item.get("last_modified"),
        "size": item.get("size"),
        "etag": item.get("etag"),
        "contentType": item.get("content_type"),
        "content": item.get("content"),
    }
    
    # Ingest the article into Weaviate
    client.data_object.create(
        data_object=article_data,
        class_name="NotionArticle"
    )
    print(f"Article '{article_data['objectName']}' ingested successfully.")

Article 'Notion/Development Hub.md' ingested successfully.
Article 'Notion/Development Hub/AWS.md' ingested successfully.
Article 'Notion/Development Hub/AWS/Untitled Database/API Gateway Commands.md' ingested successfully.
Article 'Notion/Development Hub/AWS/Untitled Database/Create an Lambda Function and API.md' ingested successfully.
Article 'Notion/Development Hub/AWS/Untitled Database/Lambda CLI Commands.md' ingested successfully.
Article 'Notion/Development Hub/AWS/Untitled Database/SAM CLI Template for Lambda.md' ingested successfully.
Article 'Notion/Development Hub/AWS/Untitled Database/Untitled.md' ingested successfully.
Article 'Notion/Development Hub/Blender.md' ingested successfully.
Article 'Notion/Development Hub/Blender/Projects/IMG⇒Blender using Python.md' ingested successfully.
Article 'Notion/Development Hub/Blender/Projects/IMG⇒Lasso⇒Extrude.md' ingested successfully.
Article 'Notion/Development Hub/Blender/Projects/Untitled.md' ingested successfully.
Article 'Notio

Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/Data Science Projects/Remote Pentesting Architecture.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/Data Science Projects/Remote Pentesting Architecture/Generate Filesystem for project.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/Data Science Projects/Untitled.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/AI-Assisted DevOps Toolkit”.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/API-Enum GraphQL.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/AR Character from Video  (Stable Diffusion + EBSynth).md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/AR Character from Video  (Stable Diffusion + 

Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/BugBounty-Dgraph.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/CDAPHOTOGRAPHY Vercel Nextjs13.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/CSV Data for Canva Videos.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/CSV Data for Canva Videos/David says….md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/CSV Data for Canva Videos/GPT Response 2.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/CSV Data for Canva Videos/GPT Response.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/CSV Data for Canva Videos/Inserted example code.md' ingested successfully.
Arti

Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/KnowledgeBase.repo- Where my deployments come together/KnowledgeBaseScrum/Sanity.Cdaprod.dev - Sanity.io Datasets.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/KnowledgeBase.repo- Where my deployments come together/KnowledgeBaseScrum/Slack Channel.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/KnowledgeBase.repo- Where my deployments come together/KnowledgeBaseScrum/Supabase.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/KnowledgeBase.repo- Where my deployments come together/KnowledgeBaseScrum/Untitled.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/KnowledgeBase.repo- Where my deployments come together/KnowledgeBaseScrum/Video-Metadata RAGS.md' ingested successfully

Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/Malicious Botnet/Development Guide - Frameworks, Function Classes,/9.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/Malicious Botnet/Development Guide - Frameworks, Function Classes,/Untitled.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/Malicious Botnet/Frameworks Used.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/Malicious Botnet/GPT Response.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/Malicious Botnet/How to Dev with Roadmap.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorment/My Creativity/Malicious Botnet/Inserted example code.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Dev Enviorme

...

Article 'Notion/Development Hub/DevelopmentHubTable/Python/Llama-Index/Index Sanity ⇒ BlogPosts Schema.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Python/Llama-Index/My Data.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Python/Llama-Index/My Query Engines.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Python/Llama-Index/Nodes - Chunking.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Python/Llama-Index/Untitled.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Python/Llama-Index/Weaviate - Llama Index.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Python/Llama-Index/airtable.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Python/Llama-Index/ipynb.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Python/Llama-Index/llama_index_tool(langcha

Article 'Notion/Development Hub/DevelopmentHubTable/Python/Untitled 3.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Python/Untitled.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Python/User Persona Generation (GPT4).md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Python/Weaviate Snippets.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Python/Weaviate Snippets/WeaviateSnippets/ChatVectorDB.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Python/Weaviate Snippets/WeaviateSnippets/Curl - Batch Import.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Python/Weaviate Snippets/WeaviateSnippets/Curl - Create Schema Class.md' ingested successfully.
Article 'Notion/Development Hub/DevelopmentHubTable/Python/Weaviate Snippets/WeaviateSnippets/Data Agent.md' ingested successfully.
Article 'Notion/Development Hub/

Article 'Notion/Development Hub/Generated About Me/Study timeline.md' ingested successfully.
Article 'Notion/Development Hub/Generated About Me/Traits that I found important when reflecting.md' ingested successfully.
Article 'Notion/Development Hub/Generated About Me/Untitled.md' ingested successfully.
Article 'Notion/Development Hub/Generated About Me/Website (CyberDev).md' ingested successfully.
Article 'Notion/Development Hub/Generated About Me/Where I am going and how I will get there, by Phind.md' ingested successfully.
Article 'Notion/Development Hub/Generated About Me/WhoAmI Aiming to Become.md' ingested successfully.
Article 'Notion/Development Hub/Generated About Me/WhoAmI Aiming to Become/Positions with Pay (as of 2021).md' ingested successfully.
Article 'Notion/Development Hub/Generated About Me/one word naming.md' ingested successfully.
Article 'Notion/Development Hub/Hardware.md' ingested successfully.
Article 'Notion/Development Hub/Hardware/Untitled Database/Camera motor

Article 'Notion/Development Hub/Self-Taught/BuiltApps/Langchain/Examples/Langchain Cookbook/Cells from the Notebook/Embedding Text.md' ingested successfully.
Article 'Notion/Development Hub/Self-Taught/BuiltApps/Langchain/Examples/Langchain Cookbook/Cells from the Notebook/Example Selectors.md' ingested successfully.
Article 'Notion/Development Hub/Self-Taught/BuiltApps/Langchain/Examples/Langchain Cookbook/Cells from the Notebook/Importing Langchain Modules.md' ingested successfully.
Article 'Notion/Development Hub/Self-Taught/BuiltApps/Langchain/Examples/Langchain Cookbook/Cells from the Notebook/Paste md.md' ingested successfully.
Article 'Notion/Development Hub/Self-Taught/BuiltApps/Langchain/Examples/Langchain Cookbook/Cells from the Notebook/Prompt Templates.md' ingested successfully.
Article 'Notion/Development Hub/Self-Taught/BuiltApps/Langchain/Examples/Langchain Cookbook/Cells from the Notebook/SimpleSequentialChain.md' ingested successfully.
Article 'Notion/Development Hub/S

Article 'Notion/Development Hub/Startup Ideas/Pitch to Pioneer- Share Your Startup Idea with David Cannan (Cdaprod).md' ingested successfully.
Article 'Notion/Development Hub/Startup Ideas/Untitled 2.md' ingested successfully.
Article 'Notion/Development Hub/Startup Ideas/Untitled.md' ingested successfully.


That's fantastic! It looks like your articles have been successfully ingested into Weaviate. With your data now in Weaviate, you're set to explore some of its powerful features, such as semantic search, automatic classification, and more.

### Next Steps

Here are a few suggestions on what you can do next:

#### 1. **Semantic Search on Your Articles**
Leverage Weaviate's vector search capabilities to perform semantic searches across your articles. This allows you to find articles based on the meaning and context of the search query rather than just keyword matching.

In [10]:
query_result = client.query.get(
    "Article",
    ["title", "content"]
).with_near_text({
    "concepts": ["integrations"]
}).do()

for article in query_result['data']['Get']['Article']:
    print(f"Title: {article['title']}")
    print(f"Content: {article['content'][:200]}...")  # Print the first 200 characters
    print("-----------")

Title: None
Content: For interactionless use, set `ALLOWLISTED_PLUGINS=example-plugin1,example-plugin2,example-plugin3` in your `.env`

|   |   |   |
|---|---|---|
|Plugin|Description|Location|
|Twitter|AutoGPT is capable...
-----------
Title: None
Content: ---
Status: Done
Content-Type: Section/Bullet Points
Field of Study:
  - Artifical Intelligence Engineer
  - Backend Developer
  - Cloud Architecture Engineering
  - Cyber Security Engineering
  - Dat...
-----------
Title: None
Content: - Database
    
    Something that converts to json (couchdb, graphQL, noSQL)
    
- User Interface
    
    Dashboard/Landing Page
    
- Takes input
    
      
    
- Processes Input
- No REST API...
-----------
Title: None
Content: ...
-----------
Title: None
Content: ---
Status: Not started
Content-Type: Block Paragraph
Field of Study:
  - Artifical Intelligence Engineer
  - Cloud Architecture Engineering
  - Data Engineering
Assign: David Cannan
---
As a speciali...
-----------
Title: None
Cont

To generate titles for all Article objects that currently don't have titles using available Weaviate modules programmatically within one cell, you would ideally leverage a module like generative-openai for content generation. However, as of my last update, directly invoking generative AI models to automatically perform such tasks through the Weaviate Python client isn't supported out of the box. This process typically involves external processing or manual intervention.

Despite this, I'll guide you through a conceptual approach that outlines how you might achieve this by integrating external AI/ML APIs (such as OpenAI's GPT-3) with Weaviate's capabilities. This involves fetching articles without titles, generating titles using an AI model, and then updating those articles in Weaviate.

Conceptual Approach:

Fetch Articles Without Titles: Your existing query fetches articles based on a concept search. To specifically target articles without titles, you might need to adjust this query or perform additional filtering in Python.
Generate Titles with an AI Model: Use an external API like OpenAI's GPT-3 to generate titles. You would iterate over your fetched articles, sending content to the API as prompts for title generation.
Update Articles in Weaviate: Once you have the generated titles, update each corresponding article in Weaviate with its new title.


In [11]:
from openai import OpenAI
import weaviate

client = weaviate.Client("http://cda-DESKTOP:8080")

# Initialize OpenAI client to use LM Studio locally
openai_client = OpenAI(base_url="http://localhost:9999/v1", api_key="not-needed")

# Fetch articles lacking titles
query_result = client.query.get("Article", ["title", "content"]).with_near_text({"concepts": ["integrations"]}).do()

for article in query_result['data']['Get']['Article']:
    if article['title'] is None:
        # Use local LM to generate a title
        completion = openai_client.chat.completions.create(
            model="local-model",  # Specify the local engine you have
            messages=[
                {"role": "system", "content": "Generate a concise title."},
                {"role": "user", "content": article['content'][:200]}
            ],
            temperature=0.7
        )
        generated_title = completion.choices[0].message["content"]

        # Update the article in Weaviate with the new title
        update_uuid = article['_additional']['id']
        client.data_object.update(
            data_object={"title": generated_title},
            class_name="Article",
            uuid=update_uuid
        )
        print(f"Updated Article UUID: {update_uuid} with Title: {generated_title}")

C:\Users\david\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\weaviate\warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


APIConnectionError: Connection error.

# 2. **Data Aggregation**
Aggregate data to get insights, such as the count of articles per tag, or the most recent articles.

In [ ]:
# Assuming 'client' is already instantiated Weaviate client
try:
    aggregation_result = client.query.aggregate(
        class_name="Tag"
    ).group_by(
        "name"  # Correct way to specify the property to group by
    ).aggregate(
        "count"  # Specifies the aggregate function directly
    ).do()

    for group in aggregation_result["data"]["Aggregate"]["Tag"]["groups"]:
        tag_name = group["groupedBy"]
        articles_count = group["aggregates"][0]["value"]
        print(f"Tag: {tag_name}, Articles: {articles_count}")

except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
response = (
    client.query
    .aggregate("Article")
    .with_meta_count()
    .do()
)
print(json.dumps(response, indent=2))

In [ ]:
response = (
    client.query
    .aggregate("Article")
    .with_group_by_filter(["Tag"])
    .with_fields("groupedBy { value }")
    .with_meta_count()
    .do()
)
print(json.dumps(response, indent=2))

In [ ]:
response = (
    client.query
    .aggregate("NotionArticle")
    .with_fields("title { count }")
    .do()
)
print(json.dumps(response, indent=2))

#### 3. **Update and Manage Your Data**
As your data evolves, you might want to update articles or tags, add new ones, or delete outdated information.

- **Update an Article**:

```
# Assuming you have the UUID of the article you want to update

client.data_object.update(
    uuid="the-article-uuid",
    data_object={"content": "Updated content..."},
    class_name="Article"
)
```

- **Delete an Article**:

```
# client.data_object.delete(
#     uuid="the-article-uuid",
#     class_name="Article"
# )
```

#### 4. **Explore Advanced Queries**
Weaviate supports GraphQL for complex queries, allowing you to deeply explore your data with filters, references, and more.

### Further Exploration
- **Classification**: Automatically categorize your articles into predefined tags using Weaviate's classification module.
- **Custom Modules**: If you have specific needs, look into creating custom Weaviate modules.

### Documentation and Community
- For detailed examples, the [Weaviate documentation](https://weaviate.io/developers/weaviate/current/) is an invaluable resource.
- If you encounter challenges or have questions, the Weaviate community on [GitHub](https://github.com/semi-technologies/weaviate) and [Slack](https://join.slack.com/t/weaviate-io/shared_invite/zt-t24h9jed-~5~p9mR~_FpN_~7ZlQJ6jw) can provide support and guidance.

Congratulations on setting up your knowledge graph with Weaviate! This is just the beginning of unlocking the potential of your data through semantic technology.

# Extending the Available Schema with `technical_schema_classes` with `Complex Properties`

In [ ]:
technical_schema_classes = {
  "classes": [
    {
      "class": "SourceCode",
      "description": "A class to store source code objects.",
      "properties": [
        {
          "name": "id",
          "dataType": ["string"],
          "description": "Unique identifier for the source code object."
        },
        {
          "name": "imports",
          "dataType": ["string[]"],
          "description": "List of extracted required packages."
        },
        {
          "name": "classes",
          "dataType": ["string[]"],
          "description": "List of extracted classes from the code."
        },
        {
          "name": "code",
          "dataType": ["text"],
          "description": "Source code snippets."
        },
        {
          "name": "syntax",
          "dataType": ["string"],
          "description": "The programming language syntax/extension."
        },
        {
          "name": "context",
          "dataType": ["text"],
          "description": "Any extracted text, markdown, comments, or docstrings."
        },
        {
          "name": "metadata",
          "dataType": ["string"],
          "description": "Extracted or generated metadata tags."
        }
      ]
    },
    {
      "class": "Table",
      "description": "A class to store tables.",
      "properties": [
        {
          "name": "headers",
          "dataType": ["string[]"],
          "description": "Headers of the table."
        },
        {
          "name": "rows",
          "dataType": ["string[]"],
          "description": "Rows of the table, each row being a dictionary."
        }
      ]
    },
    {
      "class": "MarkdownDocument",
      "description": "A class to store markdown documents.",
      "properties": [
        {
          "name": "metadata",
          "dataType": ["string"],
          "description": "Metadata of the document."
        },
        {
          "name": "tables",
          "dataType": ["Table[]"],
          "description": "List of tables in the document."
        },
        {
          "name": "code_blocks",
          "dataType": ["SourceCode[]"],
          "description": "List of code blocks in the document."
        },
        {
          "name": "content",
          "dataType": ["text"],
          "description": "The textual content of the document."
        },
        {
          "name": "blob_data",
          "dataType": ["blob"],
          "description": "Binary data for images or videos in the document."
        }
      ]
    }
  ]
}

class SchemaManager:
    def _get_dict_from_object(self, schema):
        if isinstance(schema, dict):
            return schema
        elif isinstance(schema, str):
            import json
            return json.loads(schema)
        else:
            raise TypeError("Unsupported schema format")

    def _create_classes_with_primitives(self, classes):
        # Example logic to handle class creation with primitive data types
        for class_def in classes:
            print(f"Creating class {class_def['class']} with primitives")
            # Add logic here to create class in Weaviate with primitive properties

    def _create_complex_properties_from_classes(self, classes):
        # Example logic to add complex properties to classes
        for class_def in classes:
            print(f"Adding complex properties to {class_def['class']}")
            # Add logic here to add complex properties like references

    def create_schema(self, schema):
        loaded_schema = self._get_dict_from_object(schema)
        self._create_classes_with_primitives(loaded_schema["classes"])
        self._create_complex_properties_from_classes(loaded_schema["classes"])

schema_manager = SchemaManager()
schema_manager.create_schema(technical_schema_classes)

In [ ]:
print(schema_manager)



In [14]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(template="{schema}{technical}", input_variables=[schema, technical_schema_classes])
partial_prompt = prompt.partial(schema=schema)
print(partial_prompt.format(schema="baz"))

KeyError: 'input_variables'